# Stream Parquet Files to Delta Table
This notebook reads Parquet files as a stream and writes them to a Delta table with schema evolution enabled.

In [ ]:
# Configure spark to allow schema merging
spark.conf.set("spark.sql.streaming.schemaInference", True)
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", True)

In [ ]:
# Read the streaming parquet files
stream_df = spark.readStream.format("parquet") \
    .option("mergeSchema", "true") \
    .option("schemaLocation", "abfss://external-location@databricksdevstgacc.dfs.core.windows.net/schema/stream_test") \
    .load("abfss://external-location@databricksdevstgacc.dfs.core.windows.net/files2/")

In [ ]:
# Write the stream to a delta table
checkpoint_path = "abfss://external-location@databricksdevstgacc.dfs.core.windows.net/checkpoints/stream_test"

stream_query = stream_df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", checkpoint_path) \
    .option("mergeSchema", "true") \
    .trigger(processingTime='5 seconds') \
    .toTable("dev_catalog.default.stream_test")

In [ ]:
# Wait for the stream to complete
stream_query.awaitTermination()